In [8]:
import os
import uuid
from typing import List

import requests
from bs4 import BeautifulSoup
import pandas as pd
import psycopg2
from sqlalchemy import exc, create_engine

# CREATE INDEX index_name
# ON table_name (column1_name, column2_name);

In [92]:
def sql_connection(rds_schema: str):
    """
    SQL Connection function connecting to my postgres db with schema = nba_source where initial data in ELT lands.
    Args:
        rds_schema (str): The Schema in the DB to connect to.
    Returns:
        SQL Connection variable to a specified schema in my PostgreSQL DB
    """
    RDS_USER = os.environ.get("RDS_USER")
    RDS_PW = os.environ.get("RDS_PW")
    RDS_IP = os.environ.get("IP")
    RDS_DB = os.environ.get("RDS_DB")
    try:
        connection = create_engine(
            f"postgresql+psycopg2://{RDS_USER}:{RDS_PW}@{RDS_IP}:5432/{RDS_DB}",
            connect_args={"options": f"-csearch_path={rds_schema}"},
            # defining schema to connect to
            echo=False,
        )
        print(f"SQL Connection to schema: {rds_schema} Successful")
        return connection
    except exc.SQLAlchemyError as e:
        return e

def to_sql_upsert(conn, table_name: str, df: pd.DataFrame, table_type: str):
    """
    SQL Table function to upsert a Pandas DataFrame into a SQL Table.

    Will create a new table if it doesn't exist.  If it does, it will insert new records and upsert new column values onto existing records (if applicable).

    You have to do some extra index stuff to the pandas df to specify what the primary key of the records is (this data does not get upserted).

    Args:
        conn (SQL Connection): The connection to the SQL DB.

        table_name (str): The Table name to write to SQL as.

        df (DataFrame): The Pandas DataFrame to store in SQL

        table_type (str): A placeholder which should always be "upsert"

    Returns:
        Writes the Pandas DataFrame to a Table in Snowflake in the {nba_source} Schema we connected to.

    """
    try:
        # If the table does not exist, we should just use to_sql to create it - watchout for table_schema
        if not conn.execute(
            f"""SELECT EXISTS (
                SELECT FROM information_schema.tables 
                WHERE  table_schema = 'nba_source_dev' 
                AND    table_name   = '{table_name}');
                """
        ).first()[0]:
            df.to_sql(table_name, conn)
            return True

        # If it already exists...
        temp_table_name = f"temp_{uuid.uuid4().hex[:6]}"
        df.to_sql(temp_table_name, conn, index=True)

        index = list(df.index.names)
        index_sql_txt = ", ".join([f'"{i}"' for i in index])
        columns = list(df.columns)
        headers = index + columns
        headers_sql_txt = ", ".join(
            [f'"{i}"' for i in headers]
        )  # index1, index2, ..., column 1, col2, ...
        # col1 = exluded.col1, col2=excluded.col2
        # this is excluding the primary key columns needed to identify the unique rows.
        update_column_stmt = ", ".join([f'"{col}" = EXCLUDED."{col}"' for col in columns])

        # For the ON CONFLICT clause, postgres requires that the columns have unique constraint
        query_pk = f"""
        ALTER TABLE "{table_name}" DROP CONSTRAINT IF EXISTS unique_constraint_for_upsert;
        ALTER TABLE "{table_name}" ADD CONSTRAINT unique_constraint_for_upsert UNIQUE ({index_sql_txt});
        """

        conn.execute(query_pk)

        # Compose and execute upsert query
        query_upsert = f"""
        INSERT INTO "{table_name}" ({headers_sql_txt}) 
        SELECT {headers_sql_txt} FROM "{temp_table_name}"
        ON CONFLICT ({index_sql_txt}) DO UPDATE 
        SET {update_column_stmt};
        """
        conn.execute(query_upsert)
        conn.execute(f"DROP TABLE {temp_table_name}")
        print(f"SQL Upsert Function Successful, {len(df)} records added or upserted into {table_name}")
        pass
    except BaseException as error:
        print(f"SQL Upsert Function Failed for {table_name} ({len(df)} rows), {error}")

In [9]:
def schedule_scraper(year: str, month_list: List[str]) -> pd.DataFrame:
    """
    Web Scrape Function to scrape Schedule data by iterating through a list of months
    Args:
        year (str) - The year to scrape
        month_list (list) - List of full-month names to scrape
    
    Returns:
        DataFrame of Schedule Data to be stored.
    
    """
    try:
        schedule_df = pd.DataFrame()
        completed_months = []
        for i in month_list:
            url = f"https://www.basketball-reference.com/leagues/NBA_{year}_games-{i}.html"
            html = requests.get(url).content
            soup = BeautifulSoup(html, "html.parser")

            headers = [th.getText() for th in soup.findAll("tr")[0].findAll("th")]
            headers[6] = "boxScoreLink"
            headers[7] = "isOT"
            headers = headers[1:]

            rows = soup.findAll("tr")[1:]
            date_info = [
                [th.getText() for th in rows[i].findAll("th")] for i in range(len(rows))
            ]

            game_info = [
                [td.getText() for td in rows[i].findAll("td")] for i in range(len(rows))
            ]
            date_info = [i[0] for i in date_info]

            schedule = pd.DataFrame(game_info, columns=headers)
            schedule["Date"] = date_info

            print(
                f"Schedule Function Completed for {i}, retrieving {len(schedule)} rows"
            )
            completed_months.append(i)
            schedule_df = schedule_df.append(schedule)

        schedule_df = schedule_df[
            ["Start (ET)", "Visitor/Neutral", "Home/Neutral", "Date"]
        ]
        schedule_df["proper_date"] = pd.to_datetime(schedule_df["Date"]).dt.date
        schedule_df.columns = schedule_df.columns.str.lower()
        schedule_df = schedule_df.rename(
            columns={
                "start (et)": "start_time",
                "visitor/neutral": "away_team",
                "home/neutral": "home_team",
            }
        )

        print(
            f"Schedule Function Completed for {' '.join(completed_months)}, retrieving {len(schedule_df)} total rows"
        )
        return schedule_df
    except IndexError as index_error:
        print(
            f"{i} currently has no data in basketball-reference, stopping the function and returning data for {' '.join(completed_months)}"
        )
        schedule_df = schedule_df[
            ["Start (ET)", "Visitor/Neutral", "Home/Neutral", "Date"]
        ]
        schedule_df["proper_date"] = pd.to_datetime(schedule_df["Date"]).dt.date
        schedule_df.columns = schedule_df.columns.str.lower()
        schedule_df = schedule_df.rename(
            columns={
                "start (et)": "start_time",
                "visitor/neutral": "away_team",
                "home/neutral": "home_team",
            }
        )
        return schedule_df
    except BaseException as e:
        print(f"Schedule Scraper Function Failed, {e}")
        df = []
        return df

schedule_df = schedule_scraper("2022", ["april", "may", "june"])

Schedule Function Completed for april, retrieving 129 rows
Schedule Function Completed for may, retrieving 40 rows
june currently has no data in basketball-reference, stopping the function and returning data for april may


In [4]:
conn = sql_connection('nba_source_dev')

SQL Connection to schema: nba_source_dev Successful


In [90]:
dummy_schedule1 = pd.read_csv('dummy_schedule1.csv')
dummy_schedule2 = pd.read_csv('dummy_schedule2.csv')
dummy_schedule1['proper_date'] = pd.to_datetime(dummy_schedule1['proper_date'])
dummy_schedule2['proper_date'] = pd.to_datetime(dummy_schedule2['proper_date'])

dummy_schedule1 = dummy_schedule1.set_index(['away_team', 'home_team', 'proper_date'])
dummy_schedule1 = dummy_schedule1.rename_axis(['away_team', 'home_team', 'proper_date'])

dummy_schedule2 = dummy_schedule2.set_index(['away_team', 'home_team', 'proper_date'])
dummy_schedule2 = dummy_schedule2.rename_axis(['away_team', 'home_team', 'proper_date'])

In [77]:
dummy_schedule1.index.names

FrozenList(['away_team', 'home_team', 'proper_date'])

In [91]:
to_sql_upsert(conn, 'nba_schedule', dummy_schedule2, 'append')

True